# Laboratorio 3 - Data Science
Diego Morales - 21146   
Alejandro Ortega - 

In [514]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import os
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow.keras import layers, models


In [515]:
print(f"Tensor flow version: {tf.__version__}")

Tensor flow version: 2.10.0


## 1. Preparación de datos

In [516]:
# Cargar el archivo .p
with open('Datos_Rotulos_Trafico/entrenamiento.p', 'rb') as file:
    data = pickle.load(file)

# Imprimir las claves disponibles
print(data.keys())

# Inspeccionar un poco más el contenido
for key in data.keys():
    print(f"Key: {key}, Type: {type(data[key])}, Length: {len(data[key])}")

dict_keys(['coords', 'labels', 'features', 'sizes'])
Key: coords, Type: <class 'numpy.ndarray'>, Length: 34799
Key: labels, Type: <class 'numpy.ndarray'>, Length: 34799
Key: features, Type: <class 'numpy.ndarray'>, Length: 34799
Key: sizes, Type: <class 'numpy.ndarray'>, Length: 34799


In [517]:
def load_data(pickle_file):
    with open(pickle_file, 'rb') as file:
        data = pickle.load(file)
    return data['features'], data['labels']

In [518]:
X_train, y_train = load_data('Datos_Rotulos_Trafico/entrenamiento.p')
X_validation, y_validation = load_data('Datos_Rotulos_Trafico/validacion.p')
X_test, y_test = load_data('Datos_Rotulos_Trafico/prueba.p')

In [519]:
X_train = X_train / 255.0
X_validation = X_validation / 255.0
X_test = X_test / 255.0


y_train = np.array(y_train)
y_validation = np.array(y_validation)
y_test = np.array(y_test)

In [520]:
X_train.shape

(34799, 32, 32, 3)

In [521]:
y_train.shape

(34799,)

In [522]:
X_validation.shape

(4410, 32, 32, 3)

In [523]:
y_validation.shape

(4410,)

In [524]:
X_test.shape

(12630, 32, 32, 3)

In [525]:
y_test.shape

(12630,)

## 2. Implementación de la arquitectura Le-Net

## 3. Contrucción de modelo

In [526]:
model = models.Sequential()

In [527]:
# Definir la estructura de capas convolucionales, capas de pooling y capas fully connected
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(43, activation='softmax'))


In [528]:
model.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_69 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_70 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_71 (Conv2D)          (None, 4, 4, 64)          36928     
                                                                 
 flatten_28 (Flatten)        (None, 1024)            

In [529]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

## 4. Entrenamiento del modelo

In [530]:
# Mostrar cómo cargar los datos de entrenamiento y validación en lotes.
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(len(X_train)).batch(128)

validation_dataset = tf.data.Dataset.from_tensor_slices((X_validation, y_validation))
validation_dataset = validation_dataset.batch(128)

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [531]:
history = model.fit(train_dataset, epochs=5, validation_data=validation_dataset)

Epoch 1/5
 87/272 [========>.....................] - ETA: 37s - loss: 3.2737 - accuracy: 0.2112

KeyboardInterrupt: 

In [ ]:
print(history.history.keys())

In [ ]:
print(f"Accuracy: {history.history['accuracy']}")